<a href="https://colab.research.google.com/github/ravishankarramakrishnan/Deep_Learning_Practice/blob/master/1%20-%20Sentiment_Analysis_CNN_DeepNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Libraries

import numpy as np
import pandas as pd
import re
import math
import time
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from google.colab import drive

Using TensorFlow backend.


In [0]:
# Import Deep Learning Libraries

try:
    %tensorflow_version 2.x
except:
    pass

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [0]:
# Data Preprocessing

# Loading the Data file

drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Loading the Training and Test Set

cols = ["sentiment", "id", "date", "query", "user", "text"]

# Training Set

train_data = pd.read_csv("/content/drive/My Drive/Mordern NLP - Python/Sentiment Analysis - CNN and NLP/training.1600000.processed.noemoticon.csv",
                         header = None,
                         names = cols,
                         engine = "python",
                         encoding = "latin1")

# Test Data

test_data = pd.read_csv("/content/drive/My Drive/Mordern NLP - Python/Sentiment Analysis - CNN and NLP/testdata.manual.2009.06.14.csv",
                         header = None,
                         names = cols,
                         engine = "python",
                         encoding = "latin1")

In [0]:
# Check the Dataset

test_data.tail(3)

,sentiment,id,date,query,user,text
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."
497,0,14076,Sun Jun 14 21:36:17 UTC 2009,iran,captain_pete,Reading the tweets coming out of Iran... The w...


In [0]:
train_data.head(2)

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [0]:
# Data Preprocessing

# Data Cleaning

data = train_data.copy()

# Drop Unwanted rows

data.drop(["id", "date", "query", "user"], axis = 1, inplace = True)

In [0]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [0]:
# Define a Function to Clean all the Tweets

def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Replace any words (+) followed after @
    tweet = re.sub(r"@[A-Za-z0-9]+", " ", tweet)
    # Replace Http (s is optional so ?, followed by [a-Z0-9] and + for multiple times and ./)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", " ", tweet)
    # Remove Extra Whitespaces
    tweet = re.sub(r" +", " ", tweet)
    return tweet

In [0]:
# Identify and See the Target Variables

data_labels = data.sentiment.values
print(set(data_labels))

{0, 4}


In [0]:
# We Need to Change 4 to 1, So we Manually Impute it
data_labels[data_labels == 4] = 1

In [0]:
set(data_labels)

{0, 1}

In [0]:
# Now we Apply the Cleantweet Method to Text

start = time.time()
data_clean = [clean_tweet(tweet) for tweet in data.text]
stop = time.time()
duration = stop-start
print(duration)

265.7469401359558


In [0]:
# Building a Tokeniser

start = time.time()
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size = 2**16)
stop = time.time()
duration = stop-start
print(duration)

734.7560949325562


In [0]:
# Tokenize the Data Inputs 

start = time.time()
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]
stop = time.time()
duration = stop-start
print(duration)

616.9548978805542


In [0]:
# Padding

MAX_LEN = max([len(sentence) for sentence in data_inputs])

data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                           value = 0,
                                                           padding = "post",
                                                           maxlen = MAX_LEN)

In [0]:
type(data_inputs)

numpy.ndarray

In [0]:
import pickle
#pickle.dump(data_inputs, open("data_inputs_pkl.pkl", "wb"))

In [0]:
#from google.colab import files
#files.download('data_inputs_pkl.pkl')

In [0]:
# Read a Pickle file

#pickle.load("/content/drive/My Drive/Mordern NLP - Python/Sentiment Analysis - CNN and NLP/data_inputs_pkl.pkl")

with open('/content/drive/My Drive/Mordern NLP - Python/Sentiment Analysis - CNN and NLP/data_inputs_pkl.pkl', 'rb') as file:
    data_inputs = pickle.load(file)


In [0]:
type(data_inputs)

numpy.ndarray

In [0]:
# Splitting the Data into Test Set

test_idx = np.array(np.random.randint(0, 800000, size=8000))

test_idx = np.concatenate((test_idx, test_idx+800000))

In [0]:
test_idx[:25]

array([523540, 291895, 313499, 591934,  14738, 431128, 744018, 745526,
       444903, 379131, 206570, 693672, 222428, 610149, 786086, 255242,
       365519, 513969, 144703, 371559, 688742, 348932, 499365, 253241,
       314703])

In [0]:
len(test_idx)

16000

In [0]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]

In [0]:
train_inputs = np.delete(data_inputs, test_idx, axis = 0)
train_labels = np.delete(data_labels, test_idx, axis = 0)

In [0]:
# Get the Shape

test_inputs.shape, test_labels.shape, train_inputs.shape, train_labels.shape

((16000, 273), (16000,), (1584064, 273), (1584064,))

In [0]:
# Model Building

# Deep Convolutional Neural Network

class DCNN(tf.keras.Model):

    def __init__(self, vocab_size, emb_dim=128, nb_filters = 50, FFN_units = 512, nb_classes = 2,
                 dropout_rate = 0.1, training = False, name = "dcnn"):
      
        super(DCNN, self).__init__(name = name)

        self.embedding = layers.Embedding(vocab_size, emb_dim)

        # Get the First Conv Layer with 2 kernel
        self.bigram = layers.Conv1D(filters= nb_filters, kernel_size= 2, padding= "valid", activation= "relu")

        # Perform Max Pooling 
        self.pool_1 = layers.GlobalMaxPool1D()

        # Get the Second Conv Layer with 3 kernel
        self.trigram = layers.Conv1D(filters= nb_filters, kernel_size= 3, padding= "valid", activation= "relu")

        # Perform Max Pooling 
        self.pool_2 = layers.GlobalMaxPool1D()

        # Get the Third Conv Layer with 4 kernel
        self.fourgram = layers.Conv1D(filters= nb_filters, kernel_size= 4, padding= "valid", activation= "relu")

        # Perform Max Pooling 
        self.pool_3 = layers.GlobalMaxPool1D()

        # Add a Dense Layer

        self.dense_1 = layers.Dense(units= FFN_units, activation= "relu")

        # Add dropouts to avoud overfitting

        self.dropout = layers.Dropout(rate= dropout_rate)

        # Add Last Dense Layer and Check for the Class size - Binary or Multiclass

        if nb_classes == 2:
            self.last_dense = layers.Dense(units= 1, activation= "sigmoid")
        else:
            self.last_dense = layers.Dense(units= nb_classes, activation= "softmax")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        # Work for Bigram Model
        x_1 = self.bigram(x)
        x_1 = self.pool_1(x_1)
        # work for Trigram Model
        x_2 = self.trigram(x)
        x_2 = self.pool_1(x_2)
        # Work for Fourgram Model
        x_3 = self.fourgram(x)
        x_3 = self.pool_1(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis = -1) # (batch size, 3*nb_filters)

        merged = self.dense_1(merged)

        merged = self.dropout(merged, training)

        # Get the Output

        output = self.last_dense(merged)

        return output

In [0]:
# Creating a Config File structure - Hyperparameter Tuning

#VOCAB_SIZE = tokenizer.vocab_size
VOCAB_SIZE = 2 ** 16
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(train_labels))
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [0]:
VOCAB_SIZE

65536

In [0]:
# Training

# Create an Instance and Train it

# Define a Model

Dcnn = DCNN(vocab_size = VOCAB_SIZE, emb_dim = EMB_DIM, nb_filters = NB_FILTERS,
            FFN_units = FFN_UNITS, nb_classes = NB_CLASSES, dropout_rate = DROPOUT_RATE)

In [0]:
# Go with Class length

if NB_CLASSES == 2:
    Dcnn.compile(loss = "binary_crossentropy",
                 optimizer = "adam",
                 metrics = ["accuracy"])
    
else:
    Dcnn.compile(loss = "sparse_categorical_crossentropy",
                 optimizer = "adam",
                 metrics = ["sparse_categorical_accuracy"])

In [0]:
# Create a Checkpoint obkect to create Checkpoints

checkpoint_path = "/content/drive/My Drive/Mordern NLP - Python/Sentiment Analysis - CNN and NLP/ckpt/"

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep= 2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!")

In [0]:
import tensorflow as tf
print(tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14851338026670241166
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15180381478974756409
physical_device_desc: "device: XLA_CPU device"
]


In [0]:
# Training the Model

Dcnn.fit(train_inputs,
         train_labels,
         batch_size = BATCH_SIZE,
         epochs = NB_EPOCHS)
ckpt_manager.save()

Epoch 1/5
49502/49502 [==============================] - 3318s 67ms/step - loss: 0.3949 - accuracy: 0.8225
Epoch 2/5
49502/49502 [==============================] - 3307s 67ms/step - loss: 0.3286 - accuracy: 0.8599
Epoch 3/5
49502/49502 [==============================] - 3294s 67ms/step - loss: 0.2760 - accuracy: 0.8863
Epoch 4/5
49502/49502 [==============================] - 3283s 66ms/step - loss: 0.2220 - accuracy: 0.9112
Epoch 5/5
49502/49502 [==============================] - 3282s 66ms/step - loss: 0.1758 - accuracy: 0.9314


'/content/drive/My Drive/Mordern NLP - Python/Sentiment Analysis - CNN and NLP/ckpt/ckpt-1'

In [0]:
# Finally, We Evaluate the Model

results = Dcnn.evaluate(test_inputs, test_labels, batch_size= BATCH_SIZE)
print(results)

500/500 [==============================] - 3s 7ms/step - loss: 0.4944 - accuracy: 0.8166
[0.4943913519382477, 0.8166249990463257]


time.sleep(5000)

In [0]:
# Test for Manual Data
vals = str(input("Enter the Sentence to Classify: \n"))

tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    vals, target_vocab_size = 2**16)
val = [tokenizer.encode(val) for val in vals]

Enter the Sentence to Classify: 
Is your writing good enough? Can you persuade readers to contact you? And buy from you? Can you make your content easy to read, engaging, and inspirational?


In [0]:
Dcnn(np.array([tokenizer.encode(vals)]), training=False).numpy()

array([[0.39045188]], dtype=float32)